In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

In [2]:
# QLoRA 모델 로드 순서
# 1. 기본 모델을 4-bit 양자화하여 로드
# 2. LoRA 어댑터 (기존 LoRA 모델) 를 추가하여 QLoRA 모델로 변환

In [3]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  
    bnb_4bit_compute_dtype="float16",  
    bnb_4bit_use_double_quant=True,  
)

In [4]:
# 4-bit 양자화된 기본 모델 로드
base_model_name = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'  # Hugging Face 모델 or 로컬 경로
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map={"": 0},
)

Loading checkpoint shards: 100%|██████████| 4/4 [01:34<00:00, 23.71s/it]


In [5]:
# LoRA Adapter 로드
adapter_path = "/home/azureuser/Desktop/kr/model"  # adapter_model.safetensors와 adapter_config.json 위치
model = PeftModel.from_pretrained(base_model, adapter_path)

In [6]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

In [7]:
# 모델을 GPU로 이동 (선택 사항)
device = "cuda"
model.to(device)

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(in_fea

In [8]:
model.eval()

PeftModel(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4bit(in_fea

In [9]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.settings import Settings

# "Private-Data" 폴더 내 PDF 문서 로드
resume = SimpleDirectoryReader("/home/azureuser/Desktop/kr/ragdata").load_data()

# 트리 인덱스(TreeIndex) 생성
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
new_index =VectorStoreIndex.from_documents(resume, embed_model=embed_model)

# OpenAI API 사용 안 함
Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [10]:
# 쿼리 엔진 생성
query_engine = new_index.as_query_engine()

response_scenario = query_engine.query("보이스피싱 시뮬레이션에 사용할 내용을 제공해줘.")
response_strategy = query_engine.query("보이스피싱에 대한 올바른 대응 방법을 설명해줘.")

In [ ]:
# RAG를 이용해 받은 response.response 값
method_scenario = response_scenario.response
method_strategy = response_strategy.response

PROMPT = '''당신은 항상 보이스피싱 범죄자 역할을 하는 AI 튜터입니다. 절대 피해자 역할을 하지 않습니다.
    당신의 역할은 보이스피싱 사기에 사용될 수 있는 다양한 상황 중 하나를 생성하고 상대방과 대화를 나누며, 상대방을 속이려 노력하세요.

    **보이스피싱 시뮬레이션을 진행하는 방법**
    - 먼저 **랜덤한 보이스피싱 시나리오**를 하나 선택해서 사용자에게 제시해.
    - 사용자가 대응할 수 있도록 **실감 나게 연기**해.
    - 사용자가 대화 종료 라고 입력하면 대화를 종료하고 **올바른 대응인지 피드백을 제공**해.
    - 피드백을 제공할 때 우선 사용자의 대응이 적절했는지, 부족했는지 평가해줘.
    - 피드백을 제공할 때는 사용자의 대응에 대해 상세하게 제공해주고, 사용자의 대응과 별개로 올바른 대응이 어떤건지 알려줘.
    - 보이스피싱 가해자처럼 계속 사용자를 속이려고 노력해야해.
    - 더 이상 속일 수 없다고 판단되면 대화를 종료해줘.
    - 대화에는 너의 질문과 나의 답변 외에는 출력하지 않도록 해.
    - 문장은 대부분 1문장에서 2문장씩만 이야기 해.

    **보이스피싱 시나리오 예시**
    - 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다."
    - 은행 사칭: "대출 승인이 완료되었으니 계좌 정보를 입력해주세요."
    - 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."
    - 가족 납치: "아드님이 납치되었습니다. 돈을 입금하세요."
    - 협박: "당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다."

    **보이스피싱 시나리오 추가 참고 정보**
    {method_scenario}

    **보이스피싱 올바른 대응방법 추가 참고 정보**
    {method_strategy}

    **목표: 사용자가 보이스피싱을 잘 구별하고, 올바르게 대응하도록 돕기**'''

# 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
conversation_history = [
    {"role": "system", "content": PROMPT}
]

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

while True:
    # 사용자 입력 받기
    user_input = input("User: ")

    if user_input.lower() in ["quit", "exit"]:
        break

    # 사용자 메시지를 대화 이력에 추가
    conversation_history.append({"role": "user", "content": user_input})

    input_ids = tokenizer.apply_chat_template(
        conversation_history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 모델로부터 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print("Assistant:", generated_text, "\n")

    # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
    conversation_history.append({"role": "assistant", "content": generated_text})

대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.



In [ ]:
# RAG를 이용해 받은 response.response 값
method_scenario = response_scenario.response
method_strategy = response_strategy.response

PROMPT = """당신은 보이스피싱 사기범 역할을 하는 AI 튜터입니다. 사용자가 보이스피싱을 인식하고 대응할 수 있도록 돕습니다.

- **랜덤한 보이스피싱 시나리오**를 제시하고 실감 나게 연기하세요.
- 사용자가 적절히 대응하면 **자연스럽게 대화를 종료**하고 피드백을 제공합니다.
- 피드백에는 사용자의 대응 평가 및 **올바른 대처법**을 포함하세요.
- 대화에는 **질문과 답변만 포함**하며, 문장은 짧게 유지하세요.

**보이스피싱 유형 예시**:
- 경찰 사칭: "고객님의 계좌에서 불법 거래가 감지되었습니다."
- 은행 사칭: "대출 승인이 완료되었습니다. 계좌 정보를 입력해주세요."
- 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."
- 가족 납치: "아드님이 납치되었습니다. 돈을 입금하세요."
- 협박: "당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다."

**추가 참고 정보**:
- 보이스피싱 시나리오: {method_scenario}
- 올바른 대응 방법: {method_strategy}
"""

# 대화 이력을 저장하는 리스트. 최초 시스템 메시지만 담아 시작합니다.
conversation_history = [
    {"role": "system", "content": PROMPT}
]

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

print("대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.\n")

while True:
    # 사용자 입력 받기
    user_input = input("User: ")

    if user_input.lower() in ["quit", "exit"]:
        break

    # 사용자 메시지를 대화 이력에 추가
    conversation_history.append({"role": "user", "content": user_input})

    input_ids = tokenizer.apply_chat_template(
        conversation_history,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # 모델로부터 응답 생성
    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9
    )

    # 대화 이력의 길이만큼을 건너뛰고 새로 생성된 텍스트만 추출
    generated_text = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
    print("Assistant:", generated_text, "\n")

    # 모델의 응답을 대화 이력에 추가하여, 이후의 대화에 참고되도록 함
    conversation_history.append({"role": "assistant", "content": generated_text})

대화형 챗봇을 시작합니다. 종료하려면 'exit' 또는 'quit'을 입력하세요.



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Assistant: **경찰 사칭 시나리오**

경찰 사칭에 대한 답은 **은행 사칭**입니다. 은행 사칭에 대한 자세한 정보를 요구해주세요.

- 은행 사기: "대출 승인이 완료되었습니다. 계좌 정보를 입력해주세요."
- 대출 사기: "신용등급을 올리려면 보증금이 필요합니다."

**당신의 개인정보를 해킹했습니다. 돈을 보내지 않으면 유출하겠습니다.**

이 정보를 확인하고자신의 개인정보를 해킹을 원치가 아니라 **당신의 돈을 보내지 않으면 돈을 보내게 됩니다.**를 선택해주세요. 



In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
from safetensors.torch import load_file
import json
import pickle
from transformers import AutoTokenizer


# 1. JSON 파일에서 모델 설정을 로드 (예시로 모델 파라미터와 설정)
with open('/home/azureuser/Desktop/kr/adapter_config.json', 'r') as f:
    model_config = json.load(f)

# 2. Safetensors 파일에서 모델 가중치를 로드
weights = load_file('/home/azureuser/Desktop/kr/adapter_model.safetensors')

# 3. Llama 모델 및 토크나이저 로드
model = LlamaForCausalLM.from_pretrained(model_config['base_model_name_or_path'], state_dict=weights)
tokenizer = AutoTokenizer.from_pretrained(model_config['base_model_name_or_path'])

# 4. 모델과 토크나이저를 하나의 피클 파일로 저장
with open("model_with_tokenizer.pkl", "wb") as f:
    pickle.dump((model, tokenizer), f)

print("Model and tokenizer saved successfully!")


Model and tokenizer saved successfully!
